## Breast Cancer Detection Using Deep Learning Approaches

In [1]:
import opendatasets as od
import pandas as pd

#### Load Dataset

In [2]:
od.download(
	"https://www.kaggle.com/datasets/paultimothymooney/breast-histopathology-images")

Skipping, found downloaded files in "./breast-histopathology-images" (use force=True to force download)
